<a href="https://colab.research.google.com/github/Hq030302/Sentiment-Analysis/blob/main/labelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install googletrans==3.1.0a0
!pip install transformers

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.8 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16352 sha256=ec58f8778f6edceee982f006496d4b698119367520acc1e46e5ff291d438d676
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling c

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
data = pd.read_csv("filtered_tweets_aug-sept.csv")

In [ ]:
from googletrans.client import Translator
translator = Translator()
from transformers import pipeline
sentiment_classifier = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
print(f'shape: {data.shape}')

shape: (502, 9)


In [ ]:
def clean_tweets(tweet):
  return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

In [ ]:
data['clean'] = data.apply(lambda x: clean_tweets(x['full_text']), axis=1)
print(data['clean'])
data.to_csv('clean_label_aug_sept.csv', index=True)

0      Tiang penyangganya aja ampe ga dicopot dialihf...
1                           Lrt ada aplikasi kaya krl ga
2                       eh iya ngantuk semlm salah itung
3      halo min Kartu comline ini bisa dipakai utk na...
4      Kayaknya sama sama 50 menit pas deh Mas kalau ...
                             ...                        
497        Min Raina bagu timetable per station nya dong
498    coba menhubnya suruh jadi penumpang lagi kali ...
499    Min keberangkatan disetiap stasiun setiap brp ...
500    Tolong nih orang di suspend jgn pernah boleh l...
501                                           gimana nih
Name: clean, Length: 502, dtype: object


In [ ]:
data['label'] = data['full_text'].str.encode('ascii','ignore').apply(translator.translate, dest='en')
data['label'] = data['label'].apply(getattr, args=('text',))
data['label'] = data.apply(lambda x: clean_tweets(x['label']), axis=1)

In [ ]:
print(data['label'])

0      b Even the supporting poles were not removed r...
1          b LRT is there an application like KRL or not
2                          b eh yes sleepy miscalculated
3      b hello min can this comline card be used to t...
4      b I think it s the same as 50 minutes bro if t...
                             ...                        
497      B mina raina bagu tintable per station nya dong
498    b is trying to get the Minister of Transportat...
499        b Min how many minutes do each station depart
500    b Please people who are suspended can never ri...
501                                  b how are you doing
Name: label, Length: 502, dtype: object


In [ ]:
df = (
    data
    .assign(sentiment = lambda x: x['label'].apply(lambda s: sentiment_classifier(s)))
    .assign(
        label = lambda x: x['sentiment'].apply(lambda s: (s[0]['label'])),
        score = lambda x: x['sentiment'].apply(lambda s: (s[0]['score']))
    )
)

In [ ]:
print(df)
label_jadi = df.to_csv('aug_sept_label.csv',index=True)

     Unnamed: 0                      created_at  \
0             0  Tue Sep 19 23:13:20 +0000 2023   
1             1  Tue Sep 19 22:36:09 +0000 2023   
2             2  Tue Sep 19 22:35:43 +0000 2023   
3             4  Tue Sep 19 21:22:38 +0000 2023   
4             5  Tue Sep 19 18:41:35 +0000 2023   
..          ...                             ...   
497         569  Thu Sep 14 08:21:28 +0000 2023   
498         570  Thu Sep 14 08:20:54 +0000 2023   
499         571  Thu Sep 14 08:08:33 +0000 2023   
500         572  Thu Sep 14 08:00:12 +0000 2023   
501         573  Thu Sep 14 07:56:31 +0000 2023   

                                             full_text  quote_count  \
0    @Adrianussatrio Tiang² penyangganya aja ampe g...            0   
1          @lrtjabodebek Lrt ada aplikasi kaya krl ga?            0   
2    @HeyKavindra @lrtjabodebek eh iya 🤭 ngantuk se...            0   
3    @lrtjabodebek halo min, Kartu comline ini bisa...            0   
4    @echoPiR @lrtjabodebek Kaya

In [ ]:
df